In [1]:
# Allow recursive reloading
#import builtins
#from IPython.lib import deepreload
#builtins.reload = deepreload.reload
import app
#builtins.reload(app)

Using TensorFlow backend.


## Import useful librairies

In [1]:
import numpy as np
import pandas as pd
import pickle
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from zipfile import ZipFile 

Using TensorFlow backend.


In [2]:
import app

## Import the data

We get the data from http://www.manythings.org/anki/ <br>
By change the url in the following command, it is possible to choose an other language

In [21]:
#French
!cd ./Data/raw_data && wget http://www.manythings.org/anki/fra-eng.zip

#German
!cd ./Data/raw_data && wget http://www.manythings.org/anki/deu-eng.zip

--2019-05-04 18:37:02--  http://www.manythings.org/anki/fra-eng.zip
Résolution de www.manythings.org (www.manythings.org)… 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6dc4, ...
Connexion à www.manythings.org (www.manythings.org)|104.24.109.196|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 3423204 (3,3M) [application/zip]
Enregistre : «fra-eng.zip.1»

fra-eng.zip.1       100%[===================>]   3,26M  2,06MB/s    ds 1,6s    

2019-05-04 18:37:04 (2,06 MB/s) - «fra-eng.zip.1» enregistré [3423204/3423204]

--2019-05-04 18:37:04--  http://www.manythings.org/anki/deu-eng.zip
Résolution de www.manythings.org (www.manythings.org)… 104.24.108.196, 104.24.109.196, 2606:4700:30::6818:6cc4, ...
Connexion à www.manythings.org (www.manythings.org)|104.24.108.196|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 4480305 (4,3M) [application/zip]
Enregistre : «deu-eng.zip.1»

deu-eng.zip.1       100%[===================>]   

In [11]:
# We will work with french, because we are french

archive = ZipFile('./Data/raw_data/fra-eng.zip')
donnees = pd.read_csv(archive.open('fra.txt'),
                   sep='\t',header=None, names=['english','french'])[:10000]
donnees.head()

,english,french
0,Go.,Va !
1,Hi.,Salut !
2,Run!,Cours !
3,Run!,Courez !
4,Who?,Qui ?


In [13]:
donnees.tail()

,english,french
9995,Don't overdo it.,N'en fais pas trop.
9996,Don't play dumb!,Ne fais pas l'imbécile.
9997,Don't remind me.,Ne me le rappelle pas !
9998,Don't remind me.,Ne me le rappelez pas !
9999,Don't resist us.,Ne nous résiste pas !


## Preprocessing

In [14]:
donnees['french'] = donnees["french"].apply(lambda x : '\t' + x + '\n')

The next step is to transform the sequences into numpy arrays<br>
We will use character-level one-hot encoding.<br>
We will have 3 vectors types :
     - Encoder input
     - decoder input
     - decoder labels

In [3]:
class Caracters():
    # Hight level Methods
    def __init__(self):
        self._char_to_int = dict()
        self._int_to_char = []
        self.voc_size = 0
        self.max_sentence_size = 0
        
    def fit(self, L):
        all_char = set()
        m = 0
        for s in L:
            all_char.update(list(s))
            if len(s)>m:
                m = len(s)
        self.max_sentence_size = m
        chars = sorted(list(all_char))
        self.voc_size = len(chars)
        self._int_to_char = chars
        self._char_to_int = dict([(chars[i],i) for i in range(len(chars))])
    
    def predict(self, L, fromSeqToInt = True, toTheEnd=False):        
        if fromSeqToInt:
            n = len(L)
            res = np.zeros((n, self.max_sentence_size, self.voc_size))
            for i in range(n):
                res[i] = self.vectoriseSetence(L[i],toTheEnd = toTheEnd)
        else:
            n = L.shape[0]
            res = []
            for i in range(n):
                res.append(self.decodeVec(L[i]))
                
        return res
    
    
    def save(self, fileName):
        with open(fileName,"wb") as f:
            pickle.dump(self.__dict__, f)
        
    def load(self, fileName):
        with open(fileName,"rb") as f:
            self.__dict__.update(pickle.load(f))
        
    
    
    # Low lev Methods
    def getChar(self, i):
        return self._int_to_char[i]
    
    def getInt(self, c):
        return self._char_to_int[c]
    
    def vectoriseSetence(self, sentence, toTheEnd=False):
        res = np.zeros((self.max_sentence_size, self.voc_size))
        lag = 0
        if toTheEnd:
            lag = self.max_sentence_size - len(sentence)
        for i in range(len(sentence)):
            res[i+lag][self.getInt(sentence[i])] = 1
        return res
    
    def decodeVec(self, vec):
        res = ""
        for i in range(vec.shape[0]):
            res += self.getChar(np.argmax(vec[i]))
        return res
        

In [4]:
class SequentialDecoder():
    def __init__(self, num_decoder_tokens, latent_dim, encoder_states):
        self.num_decoder_tokens = num_decoder_tokens
        self.latent_dim=latent_dim
        # Input
        self.decoder_inputs = Input(shape=(None, num_decoder_tokens))
        # LSTM
        self.decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        self.decoder_outputs, _, _ = self.decoder_lstm(self.decoder_inputs,
                                             initial_state=encoder_states)
        # Dense
        self.decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        self.decoder_outputs = self.decoder_dense(self.decoder_outputs)
        
        
        self.model = self.toPredictor()
    
    
    def toPredictor(self):
        decoder_state_input_h = Input(shape=(self.latent_dim,))
        decoder_state_input_c = Input(shape=(self.latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        
        decoder_outputs, state_h, state_c = self.decoder_lstm(
             self.decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = self.decoder_dense(decoder_outputs)
        return Model(
            [self.decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        
    def predict(self, X):
        return self.model.predict(X)
    
    def save(self, fileName):
        self.model.save_weights("./app/ML/save/" + fileName)
        
    def load(self, fileName):
        self.model.load_weights("./app/ML/save/" + fileName)

In [5]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
class SequentialEncoder():
    def __init__(self, num_encoder_tokens, latent_dim):
        self.num_encoder_tokens = num_encoder_tokens
       # Define an input sequence and process it.
        self.encoder_inputs = Input(shape=(None, num_encoder_tokens))
        # LSTM
        self.encoder = LSTM(latent_dim, return_state=True)
        self.encoder_outputs, state_h, state_c = self.encoder(self.encoder_inputs)
        # We discard `encoder_outputs` and only keep the states.
        self.encoder_states = [state_h, state_c]
        
        self.model = Model(self.encoder_inputs, self.encoder_states)
    
    def fit(self, X, y):
        self.model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
        self.model.fit(X,y)
    
    def predict(self, X):
        return self.model.predict(X)
    
    def save(self, fileName):
        self.model.save_weights("./app/ML/save/" + fileName)
        
    def load(self, fileName):
        self.model.load_weights("./app/ML/save/" + fileName)

In [6]:
class SequentialEncoderDecoder():
    def __init__(self,num_encoder_tokens, latent_dim_enc, 
                 num_decoder_tokens, latent_dim_dec):
        #Encoder
        self.encoder = SequentialEncoder(num_encoder_tokens, latent_dim_enc)
        #Decoder
        self.decoder = SequentialDecoder(num_decoder_tokens, latent_dim_dec, self.encoder.encoder_states)
        # Model for training
        self.trainModel =Model([self.encoder.encoder_inputs, self.decoder.decoder_inputs],
                      self.decoder.decoder_outputs) 
        self.trainModel.compile(optimizer='rmsprop', loss='categorical_crossentropy')
        
    def fit(self, encoder_input_data, decoder_input_data, decoder_target_data,
            batch_size=64, epochs=100,  validation_split=0.2):
        self.trainModel.fit([encoder_input_data, decoder_input_data], decoder_target_data,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_split=validation_split)
    
    def predict(self, sentences, begin_token_index ,stop_token_index, max_sentence):
        L = []
        for i in sentences.shape[0]:
            L.append(self.decodeSentence(sentences[i]), begin_token_index ,stop_token_index, max_sentence)
        return None
    
    def decodeSentence(self, sentence, begin_token_index ,stop_token_index, max_sentence):
        states_value = self.encoder.predict(input_seq)
        # Generate empty target sequence of length 1.
        target_seq = np.zeros((1, 1, self.decoder.num_decoder_tokens))
        # Populate the first character of target sequence with the start character.
        target_seq[0, 0, begin_token_index] = 1.
        stop_condition = False
        decoded_sentence = []
        while not stop_condition:
            output_tokens, h, c = decoder_model.predict(
                [target_seq] + states_value)

            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            decoded_sentence.append(sampled_token_index)

            # Exit condition: either hit max length
            # or find stop character.
            if (sampled_token_index == stop_token_index or
               len(decoded_sentence) > max_sentence):
                stop_condition = True

            # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1, self.decoder.num_decoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.

            # Update states
            states_value = [h, c]
        res = np.zeros((len(decoded_sentence),self.decoder.num_decoder_tokens))
        for r in decoded_sentence:
            res[r] = 1
        return res
        
    def save(self, fileName):
        self.encoder.save(fileName + '_enc')
        self.decoder.save(fileName + '_dec')
        
    def load(self, fileName):
        self.encoder.load(fileName + '_enc') 
        self.decoder.load(fileName + '_dec')

In [7]:
class Translator():
    def __init__(self, languageInput, languageOutput, sequentialEncoderDecoder):
        self.languageInput = languageInput
        self.languageOutput = languageOutput
        self.sequentialEncoderDecoder = sequentialEncoderDecoder
        
    def translate(self, sentance):
        vectorized = self.languageInput.predict([sentance])
        vector_translated = self.sequentialEncoderDecoder.predict(vectorized,
                                                                  begin_token_index = self.languageOutput.getInt('\t'),
                                                                  stop_token_index = self.languageOutput.getInt('\n'),
                                                                  max_sentence = self.languageOutput.max_sentence_size)
        translated = self.languageOutput.predict(vector_translated,False)

In [8]:
class Translator():
    def __init__(self, languageInput, languageOutput, sequentialEncoderDecoder):
        self.languageInput = languageInput
        self.languageOutput = languageOutput
        self.sequentialEncoderDecoder = sequentialEncoderDecoder
        
    def translate(self, sentance):
        vectorized = self.languageInput.predict([sentance])
        vector_translated = self.sequentialEncoderDecoder.predict(vectorized,
                                                                  begin_token_index = self.languageOutput.getInt('\t'),
                                                                  stop_token_index = self.languageOutput.getInt('\n'),
                                                                  max_sentence = self.languageOutput.max_sentence)
        translated = self.languageOutput.predict(vector_translated,False)

In [9]:
eng_voc = Caracters()
eng_voc.fit(donnees['english'])

NameError: name 'donnees' is not defined

In [ ]:
v = eng_voc.vectoriseSetence("Hey",True)
v

In [10]:
eng_voc.decodeVec(v)

NameError: name 'v' is not defined

In [44]:
fra_voc = Caracters()
fra_voc.fit(donnees['french'])

In [45]:
donnees['input_encoder'] = donnees["english"].apply(lambda x : eng_voc.vectoriseSetence(x, True))
donnees['input_decoder'] = donnees["french"].apply(fra_voc.vectoriseSetence)
donnees['output_decoder'] = donnees['input_decoder'].apply(lambda vec: np.vstack((vec[1:],0*vec[0].reshape((1,-1)))))

In [46]:
input_encoder = np.r_[list(donnees['input_encoder'])]
input_decoder = np.r_[list(donnees['input_decoder'])]
output_decoder =np.r_[list(donnees['output_decoder'])]

In [27]:
np.savez_compressed('Data/processed_data/fra-eng',
                    input_encoder=input_encoder,
                    input_decoder=input_decoder,
                    output_decoder=output_decoder
                   )

In [28]:
with open("app/ML/dico_eng.pickle","wb") as f:
    pickle.dump(eng_voc, f)
with open("app/ML/dico_fra.pickle","wb") as f:
    pickle.dump(fra_voc, f)

## Load the data

In [11]:
#from app.ML.src.Caracters import Caracters
fra_voc = Caracters()
eng_voc = Caracters()


eng_voc.load("app/ML/save/eng_car.pickle")
fra_voc.load("app/ML/save/fra_car.pickle") 

loaded = np.load('Data/processed_data/fra-eng.npz')

encoder_input_data = loaded['input_encoder']
decoder_input_data = loaded['input_decoder']
decoder_target_data = loaded['output_decoder']

In [12]:
eng_voc.decodeVec(encoder_input_data[1])

'             Hi.'

In [13]:
encoder_input_data[1].sum(axis=1)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.])

## Build the model

In [14]:
encoder = SequentialEncoder(eng_voc.voc_size,256)
decoder = SequentialDecoder(fra_voc.voc_size,256, encoder.encoder_states)


In [15]:
encoder_decoder = SequentialEncoderDecoder(eng_voc.voc_size,256,fra_voc.voc_size,256)

In [ ]:
encoder_decoder.fit(encoder_input_data, decoder_input_data, decoder_target_data,
            batch_size=64, epochs=10,  validation_split=0.2)
# Save model
encoder_decoder.save('model1')

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 169s 21ms/step - loss: 0.9196 - val_loss: 0.9539
Epoch 2/10
8000/8000 [==============================] - 147s 18ms/step - loss: 0.7348 - val_loss: 0.7585
Epoch 3/10
8000/8000 [==============================] - 145s 18ms/step - loss: 0.6202 - val_loss: 0.6748
Epoch 4/10
3456/8000 [===========>..................] - ETA: 1:27 - loss: 0.5741

In [17]:
model = SequentialEncoderDecoder(eng_voc.voc_size,256,fra_voc.voc_size,256)
model.load('model1')
model.fit(encoder_input_data, decoder_input_data, decoder_target_data,
            batch_size=64, epochs=10,  validation_split=0.2)

KeyboardInterrupt: 

In [22]:
model.layers[3].output

[<tf.Tensor 'lstm_2/transpose_1:0' shape=(?, ?, 256) dtype=float32>,
 <tf.Tensor 'lstm_2/while/Exit_2:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'lstm_2/while/Exit_3:0' shape=(?, 256) dtype=float32>]

In [ ]:
class SequentialEncoderDecoder():
    def __init__(self):
        self.encoder = modelEncoder()
        self.decoder = modelDecoder()
        
    
    def fit(self, encoder_input_data, decoder_input_data, decoder_target_data,
            batch_size=64, epochs=100,  validation_split=0.2):
        model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_split=validation_split)
    
    def predict(self, sentences):
        
    
    
        
    def save(self, fileName):
        self.model.save(fileName)
    def load(self, fileName):
        self.model = load_model('my_model.h5')

In [3]:
EncoderDecoder = app.ML.src.SequentialEncoderDecoder.SequentialEncoderDecoder(50,50,12,12)

ValueError: An `initial_state` was passed that is not compatible with `cell.state_size`. Received `state_spec`=[InputSpec(shape=(None, 50), ndim=2), InputSpec(shape=(None, 50), ndim=2)]; however `cell.state_size` is (12, 12)

In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
class SequentialEncoder():
    def __init__(self, num_encoder_tokens, latent_dim):
       # Define an input sequence and process it.
        self.encoder_inputs = Input(shape=(None, num_encoder_tokens))
        # LSTM
        self.encoder = LSTM(latent_dim, return_state=True)
        self.encoder_outputs, state_h, state_c = self.encoder(self.encoder_inputs)
        # We discard `encoder_outputs` and only keep the states.
        self.encoder_states = [state_h, state_c]
        
        self.model = Model(self.encoder_inputs, self.encoder_states)
    
    def fit(self, X, y):
        self.model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
        self.model.fit(X,y)
    
    def predict(self, X):
        return self.model.predict(X)
    
    def save(self, fileName):
        self.model.save_weights("" + fileName)
        
    def load(self, fileName):
        self.model.load_weights("" + fileName)

Using TensorFlow backend.


In [2]:
seqenc = SequentialEncoder(5,5)

In [3]:
seqenc.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 5)           0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 5), (None, 5), (N 220       
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________


In [4]:
seqenc.save("./app/ML/save/test")

In [5]:
seqenc.load("./app/ML/save/test")

In [6]:
seqenc.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 5)           0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 5), (None, 5), (N 220       
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________


In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun May  5 18:28:05 2019

@author: kronert
"""
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
class SequentialDecoder():
    def __init__(self, num_decoder_tokens, latent_dim, encoder_states):
        self.latent_dim=latent_dim
        # Input
        self.decoder_inputs = Input(shape=(None, num_decoder_tokens))
        # LSTM
        self.decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        self.decoder_outputs, _, _ = self.decoder_lstm(self.decoder_inputs,
                                             initial_state=encoder_states)
        # Dense
        self.decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        self.decoder_outputs = self.decoder_dense(self.decoder_outputs)
        
        
        self.model = self.toPredictor()
    
    
    def toPredictor(self):
        decoder_state_input_h = Input(shape=(self.latent_dim,))
        decoder_state_input_c = Input(shape=(self.latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        
        decoder_outputs, state_h, state_c = self.decoder_lstm(
             self.decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = self.decoder_dense(decoder_outputs)
        return Model(
            [self.decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        
    def predict(self, X):
        return self.model.predict(X)
    
    def save(self, fileName):
        self.model.save_weights("../save/" + fileName)
        
    def load(self, fileName):
        self.model.load_weights("../save/" + fileName)

In [12]:
seqdec = SequentialDecoder(5,5,seqenc.encoder_states)

In [13]:
from keras.utils import plot_model 

In [15]:
plot_model(seqdec.model, to_file='model.png')

In [17]:
import numpy as np


class SequentialEncoderDecoder():
    def __init__(self,num_encoder_tokens, latent_dim_enc, 
                 num_decoder_tokens, latent_dim_dec):
        #Encoder
        self.encoder = SequentialEncoder(num_encoder_tokens, latent_dim_enc)
        #Decoder
        self.decoder = SequentialDecoder(num_decoder_tokens, latent_dim_dec, self.encoder.encoder_states)
        # Model for training
        self.trainModel =Model([self.encoder.encoder_inputs, self.decoder.decoder_inputs],
                      self.decoder.decoder_outputs) 
        self.trainModel.compile(optimizer='rmsprop', loss='categorical_crossentropy')
        
    def fit(self, encoder_input_data, decoder_input_data, decoder_target_data,
            batch_size=64, epochs=100,  validation_split=0.2):
        self.trainModel.fit([encoder_input_data, decoder_input_data], decoder_target_data,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_split=validation_split)
    
    def predict(self, sentences):
        return None
        
    def save(self, fileName):
        self.encoder.save('enc_' + fileName)
        self.decoder.save('dec_' + fileName)
        
    def load(self, fileName):
        self.encoder.load('enc_' + fileName) 
        self.decoder.load('dec_' + fileName) 

In [18]:
seqencdec = SequentialEncoderDecoder(5,5,5,5)

In [19]:
seqencdec.trainModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, 5)      0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, None, 5)      0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, 5), (None, 5 220         input_8[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, None, 5), (N 220         input_9[0][0]                    
                                                                 lstm_4[0][1]                     
          

In [20]:
seqenc.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 5)           0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 5), (None, 5), (N 220       
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________


In [21]:
seqdec.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, 5)      0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 5), (N 220         input_5[0][0]                    
                                                                 input_6[0][0]                    
          